# Signal Classification

訊號與圖的分類一樣，在preprocess後可使用神經網路做一些AI任務，例如訊號分類、迴歸還有生成等等。

這個部分我們用音訊作為訊號的範例，來試著將聲音訊號做分類，包含以下部分:
- Audio Data Loader
- Audio Preprocess (STFT/ MFCC)
- RNN audio classification
- CNN audio classification

開始之前我們先準備一些內容。

我們使用的範例資料集是tensorflow提供的[Mini Speech Commands](https://ai.googleblog.com/2017/08/launching-speech-commands-dataset.html)資料集，從官網下載。

In [ ]:
# 下載檔案並存到data資料夾
!mkdir -p data
!wget -O data/mini_speech_commands.zip http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip
!unzip data/mini_speech_commands.zip -d data

In [ ]:
import librosa
import IPython.display as idp  # 播音工具
import librosa.display as ldp  # 畫頻譜圖工具
import numpy as np  # 輔助運算
import matplotlib.pyplot as plt  # 輔助畫圖

# import model用到的內容
import tensorflow as tf

## Audio Data Loader

與前面CNN相同，需要有data loader去對資料做讀取，而tensorflow沒有原生讀音訊的data loader (TF 2.11有，但在Colab上要另外灌TF2.11)，所以這部分要自己寫。

### 讀取音訊檔及資料切分

In [ ]:
from glob import glob  # 拿來列資料夾內容的小套件
from sklearn.model_selection import train_test_split  # 切分資料集用


def find_class(x):
    # 根據格式，找到所屬class
    return x.strip('/')[-2]


def audio_folder_datasets(dataset_path,
                          class_dictionary,
                          sr=22050,
                          duration=-1):
    # 輸入:
    #  dataset_path - 資料夾，內有數個不同class的資料夾，內有.wav檔
    #  class_dictionary - dictionary物件，對應每個資料夾的class
    #  sr -  讀取的sampling rate
    #  duration - 可指定秒數(float32)，不指定則為原檔長度
    file_names = []
    labels = []
    for cls, class_id in class_dictionary.items():
        f_list = glob(dataset_path+f'{cls}/*.wav')  # 找到該class的所有檔案
        file_names.extend(f_list)  # 加入列表
        labels.extend([class_id]*len(f_list))  # 加入相應labels
    print("total:",
          f"{len(file_names)} files of {len(class_dictionary)} classes")

    @tf.function  # 套tf.function使其可以對tensorflow tensor做動
    def load_wav(fname):
        # 使用指定sampling rate, duration讀檔
        # 這邊要用到librosa的loading才有re-sample，
        # 若已經知道每個檔案sampling rate也可以用tensorflow的tf.audio
        # 可參考:
        #  https://www.kaggle.com/code/lkergalipatak/bird-audio-classification-with-tensorflow

        x = tf.numpy_function(
            lambda x: librosa.util.fix_length(
                librosa.load(x, sr=sr, duration=duration)[0],
                size=int(sr*duration),
                mode='edge'),
            inp=[fname], Tout=tf.float32)
        return x
    def get_dataset(paths_, labels_):
        # 得到所有資料夾名稱
        path_ds = tf.data.Dataset.from_tensor_slices(paths_)  # 轉換成檔名的Dataset物件
        label_ds = tf.data.Dataset.from_tensor_slices(labels_)

        data_ds = path_ds.map(load_wav)
        return tf.data.Dataset.zip((data_ds, label_ds))

    fname_train, fname_val, label_train, label_val = train_test_split(
        file_names,
        labels,
        test_size=0.2)
    train_ds = get_dataset(fname_train, label_train)
    val_ds = get_dataset(fname_val, label_val)
    return train_ds, val_ds

生成一個dataset拿來用作基礎。

In [ ]:
# 準備一些參數輸入進function生成dataset
DATASET_PATH = 'data/mini_speech_commands/'
class_dict = {
    'down': 0,
    'go': 1,
    'left': 2,
    'no': 3,
    'right': 4,
    'stop': 5,
    'up': 6,
    'yes': 7
}
SR = 22050
DURATION = 0.8
train_ds, val_ds = audio_folder_datasets(
    DATASET_PATH,
    class_dict,
    sr=SR,
    duration=DURATION)

dataset一次丟出一個signal以及一個label

觀察data基本性質

In [ ]:
x, y = next(iter(val_ds))
print('\n', x.shape, x.numpy().min(), x.numpy().max())
print(y)

# 畫出來
ldp.waveshow(x.numpy().squeeze(), sr=SR)

# 聽看看
idp.Audio(x.numpy().squeeze(), rate=SR)

畫出一個例子

### 輸入NN前做轉換

在預處裡時使用librosa會很慢，幸好與```librosa.stft```類似，可以使用```tensorflow.signal.stft```做時頻分析，在操作的各種過程中只能用tf function來操作。

其axis為[time,frequency]，與librosa相反，使用```librosa.specshow```觀察時記得要做transpose。

為什麼要反過來是為了配合RNN的預設axis [batch,time,...]，將time擺在batch後面第一位。

若希望使用CNN模型，記得在最後多加一個空的axis，因為CNN適用的axis是[batch, hight, width, channels]。

In [ ]:
N = 512
H = 128


def get_stft(waveform):
    # 做STFT (用tensorflow得比較快)
    spectrogram = tf.signal.stft(
        waveform, frame_length=N, frame_step=H)
    # 這邊frame_length是librosa的n_fft
    #     frame_step是librosa的hop_length
    # 使用tf.signal stft出來時，單位為((timepoints-n_fft)/hop_length, n_fft/2)
    # 這是為了配合RNN等模型的

    # 取magnitude
    spectrogram = tf.abs(spectrogram)

    # 若是多加一個維度，可以用於CNN，shape (`batch_size`, `height`, `width`, `channels`).
    # spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram


# 使用STFT當作preprocess function
trian_ds_stft = train_ds.map(lambda x, y: (get_stft(x), y))\
                           .cache().shuffle(6400).prefetch(tf.data.AUTOTUNE)
val_ds_stft = val_ds.map(lambda x, y: (get_stft(x), y))\
                         .cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
# 可以看一下資料
for x_S, y in val_ds_stft:
    print(x_S.shape, x_S.numpy().min(), x_S.numpy().max())
    print(y)
    break
plt.figure(figsize=(6, 5))
ldp.specshow(x_S.numpy().T,  # 記得做transpose
             sr=SR,
             x_axis="s",
             y_axis="hz",
             cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

In [ ]:
# 跟librosa差不多，librosa有做time padding，會長一些
x_S_ = librosa.stft(x.numpy(), n_fft=N, hop_length=H)
print(x_S_.shape)
plt.figure(figsize=(6, 5))
ldp.specshow(abs(x_S_),
             sr=SR,
             x_axis="s",
             y_axis="hz",
             cmap="jet")
plt.colorbar(format="%+4.f")
plt.show()

In [ ]:
%%time
# 預讀資料，放進GPU
for x_S, y in tran_ds_stft:
    pass

這邊也可以進一步使用MFCC，但因為```tensorflow.signal```做MFCC的步驟過於複雜且須配合的指令太多，這邊我們使用librosa套tf.function做轉換

若想嘗試做tensorflow的版本可參考: https://github.com/timsainb/tensorflow2-generative-models/blob/master/7.0-Tensorflow-spectrograms-and-inversion.ipynb

In [ ]:
N_MFCC = 39


# 立一個方便的function給MFCC，套用想要的arguments，這邊記得，librosa做出來給tensorflow時需要做transpose
def mfcc_function(waveform):
    return librosa.feature.mfcc(y=waveform,
                                sr=SR,
                                n_mfcc=N_MFCC,
                                n_fft=N,
                                hop_length=H).T


@tf.function
def get_mfcc(waveform):
    # 做MFCC，因為用librosa所以要用numpy_function包起來
    mfcc = tf.numpy_function(mfcc_function, [waveform], tf.float32)
    # 若是多加一個維度，可以用於CNN，shape (`batch_size`, `height`, `width`, `channels`).
    # mfcc = mfcc[..., tf.newaxis]
    return mfcc


# 使用STFT當作preprocess function
tran_ds_mfcc = tran_ds.map(lambda x, y: (get_mfcc(x), y))\
               .cache().shuffle(6400).prefetch(tf.data.AUTOTUNE)
val_ds_mfcc = val_ds.map(lambda x, y: (get_mfcc(x), y))\
              .cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
# 可以看一下資料
for x_C, y in val_ds_mfcc:
    print(x_C.shape, x_C.numpy().min(), x_C.numpy().max())
    print(y)
    break
plt.figure(figsize=(6, 5))
ldp.specshow(x_C.numpy().T,
             sr=SR,
             x_axis="s",
             cmap="jet")  # 記得做transpose
plt.colorbar(format="%+4.f")
plt.show()

In [ ]:
%%time
# 預讀資料，放進GPU
for x_C,y in tran_ds_mfcc:
    pass

## RNN Audio classifcation

### STFT preprocess + RNN

我們可使用RNN來做對剛剛的頻譜作classification的訓練

In [ ]:
# 抓一下 data的大小
for example_spectrograms, example_spect_labels in tran_ds_stft.take(1):
    break
input_shape = example_spectrograms.shape.as_list()

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
inputs = tf.keras.Input(shape=(None, input_shape[1]))
h = layers.LSTM(256, dropout=0.1)(inputs)  # 用層LSTM
outputs = layers.Dense(len(class_dict), activation='softmax')(h)

model = models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 20
history = model.fit(
    tran_ds_stft.batch(32),
    validation_data=val_ds_stft.batch(64),
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    verbose=1
)

In [ ]:
model.evaluate(val_ds_stft.batch(64))

### MFCC preprocess + RNN

In [ ]:
# 抓一下 data的大小
for example_cepstrums, example_spect_labels in tran_ds_mfcc.take(1):
    break
input_shape = example_cepstrums.shape.as_list()

inputs = tf.keras.Input(shape=(None, input_shape[1]))  # 直接使用剛剛抓的大小
h = layers.LSTM(256, dropout=0.1)(inputs)  # 用層LSTM
outputs = layers.Dense(len(class_dict), activation='softmax')(h)

model = models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 20
history = model.fit(
    tran_ds_mfcc.batch(32),
    validation_data=val_ds_mfcc.batch(64),
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
model.evaluate(val_ds_mfcc.batch(64))

可以自行嘗試fine-tune這兩種結果。

近期語音AI論文最常用的preprocess方式可作為訊號處理AI的前處理參考:
1. 直接用一組Filter Bank
2. MFCC
3. Spectrogram

(參考: 李宏毅老師Deep Learning for Human Language Processing (2020,Spring)課程中助教統計https://youtu.be/AIKu43goh-8?list=PLJV_el3uVTsO07RpBYFsXg-bN5Lu0nhdG&t=1964)

## CNN Audio Classification

當然，因為我們已經把資料轉換成2D的頻譜了，所以也可以當作一張圖來做2D CNN。

這邊用的是STFT做時頻轉換的結果。

In [ ]:
@tf.function
def extend_dims(x, y):
    return x[..., np.newaxis], y


tran_ds_stft_ = tran_ds_stft.map(extend_dims)
val_ds_stft_ = val_ds_stft.map(extend_dims)

In [ ]:
inputs = tf.keras.Input(shape=(None, input_shape[1], 1))
h = layers.Conv2D(32, (3, 3),  activation='relu')(inputs)
h = layers.Dropout(0.1)(h)
h = layers.MaxPooling2D()(h)
h = layers.Conv2D(64, (3, 3), activation='relu')(h)
h = layers.Dropout(0.1)(h)
h = layers.MaxPooling2D()(h)
h = layers.Conv2D(64, (3, 3), activation='relu')(h)
h = layers.Dropout(0.1)(h)
h = layers.GlobalAveragePooling2D()(h)
h = layers.Flatten()(h)
h = layers.Dense(32)(h)
outputs = layers.Dense(len(class_dict), activation='softmax')(h)

model = models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 20
history = model.fit(
    tran_ds_stft_.batch(32),
    validation_data=val_ds_stft_.batch(64),
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
model.evaluate(val_ds_stft_.batch(64))

使用CNN的好處是，已經有很多CNN-based的pre-train模型可以使用來做transfer learning。

建議也可以拿transfer leraning提及的model來訓練看看!

亦可嘗試使用MFCC Preprocess後做CNN模型。

## Reference
* TF官網教學: https://www.tensorflow.org/tutorials/audio/simple_audio
* https://towardsdatascience.com/audio-augmentations-in-tensorflow-48483260b169
* https://github.com/timsainb/tensorflow2-generative-models/blob/master/7.0-Tensorflow-spectrograms-and-inversion.ipynb